In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np
import tensorboardX
import os
import sys

In [2]:
import dla34

In [3]:
heads = {'hm':3, 'dim':3, 'depthmap':1, 'orientation':8}

In [4]:
the_dla34 = dla34.DLASeg('dla34', heads, '../../model/')

In [5]:
the_dla34

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [19]:
dir(the_dla34)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_forward_unimplemented',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'base',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',

In [24]:
a = the_dla34.modules()

In [31]:
DLA = the_dla34._modules['base']

In [34]:
base_layer = DLA._modules['base_layer']

In [37]:
origin_conv = base_layer[0]

In [39]:
weights = torch.cat((origin_conv.weight, origin_conv.weight), dim=1)

In [40]:
origin_state = origin_conv.state_dict()

In [43]:
origin_state

OrderedDict([('weight',
              tensor([[[[-8.5693e-02,  6.3569e-02,  1.0894e-01,  ...,  1.5868e-01,
                          7.2469e-02, -4.7303e-02],
                        [-1.0017e-01,  7.8104e-02,  2.0329e-01,  ...,  2.5578e-01,
                          1.4172e-01, -2.3385e-02],
                        [-3.7498e-02,  1.5154e-01,  3.0601e-01,  ...,  3.2645e-01,
                          1.8902e-01, -3.8332e-03],
                        ...,
                        [ 1.1310e-01,  7.2440e-02, -1.3426e-02,  ..., -1.6752e-01,
                         -8.0464e-02,  5.4071e-03],
                        [ 1.0531e-01,  6.8242e-02, -4.4165e-02,  ..., -1.7207e-01,
                         -1.2942e-01, -2.8501e-02],
                        [ 3.4818e-03,  3.4667e-02, -6.3608e-02,  ..., -1.1538e-01,
                         -8.5321e-02, -6.0982e-02]],
              
                       [[ 9.7900e-02,  1.2798e-02, -9.8728e-02,  ..., -1.0077e-01,
                         -4.4160e-02, 

In [44]:
new_conv = nn.Conv2d(6, 16, kernel_size=7, stride=1, padding=3, bias=False)
new_conv.load_state_dict({'weight':weights})
base_layer[0] = new_conv

In [45]:
the_dla34

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(6, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [16]:
model = nn.Sequential(the_dla34)

In [18]:
model[0]

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [15]:
for i, val in enumerate(models):
    print(val)

In [9]:
dummy_input = np.random.random((1, 6, 384, 1280))
dummy_input = torch.from_numpy(dummy_input)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32)

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
from tensorboardX import SummaryWriter

In [12]:
with SummaryWriter(comment='dla34_2') as w:
    w.add_graph(the_dla34, (dummy_input,))

Given groups=1, weight of size [16, 3, 7, 7], expected input[1, 6, 384, 1280] to have 3 channels, but got 6 channels instead
Error occurs, No graph saved


RuntimeError: Given groups=1, weight of size [16, 3, 7, 7], expected input[1, 6, 384, 1280] to have 3 channels, but got 6 channels instead